In [103]:
import sys
sys.path.append('../')

In [104]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from tecniche.indicatori import RSI
from tecniche.pattern import apply_patterns
from plotting import CandlePlotting

In [105]:
df_raw = pd.read_pickle('../Data/BTC_USD_H1.pkl')

In [106]:
df_raw.shape
 

(8097, 14)

In [107]:
df_an = df_raw.iloc[-6000:].copy()
# RICORDARSI DI AZZERARE L'INDICE   
df_an.reset_index(inplace=True, drop=True)

In [108]:
df_an.shape

(6000, 14)

In [109]:
df_an = RSI(df_an)

In [110]:
df_an.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
5995,2016-12-30 19:00:00+00:00,1980,960.0,963.7,960.0,963.1,955.3,958.9,955.3,958.3,964.7,968.5,964.7,967.9,55.356060
5996,2016-12-30 20:00:00+00:00,1374,963.1,964.6,961.1,962.8,958.3,959.8,956.3,958.0,967.9,969.4,965.9,967.6,54.996099
5997,2016-12-30 21:00:00+00:00,1210,962.8,964.3,954.0,955.5,958.0,959.5,949.3,950.8,967.6,969.1,958.7,960.2,46.989034
5998,2016-12-30 22:00:00+00:00,1310,955.5,965.4,954.6,963.9,950.8,960.6,949.9,959.1,960.2,970.2,959.3,968.7,55.091404
5999,2016-12-30 23:00:00+00:00,1550,962.6,967.2,960.4,965.4,957.8,962.4,955.6,960.6,967.4,972.0,965.2,970.2,56.373709


In [111]:
df_an = apply_patterns(df_an)

In [112]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [113]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [114]:
colonne = ['time','mid_o', 'mid_h', 'mid_l', 'mid_c','ask_c', 'bid_c', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14']

In [115]:
df_slim = df_an[colonne].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(inplace=True, drop=True)

In [116]:
df_slim

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14
0,2016-04-22 21:00:00+00:00,448.6,448.6,447.9,447.9,450.1,445.7,False,-1,436.924863,53.630720
1,2016-04-22 22:00:00+00:00,447.9,447.9,447.9,447.9,450.1,445.7,False,1,437.050964,53.630720
2,2016-04-22 23:00:00+00:00,447.9,447.9,447.9,447.9,450.1,445.7,False,1,437.175425,53.630720
3,2016-04-23 00:00:00+00:00,447.9,450.1,447.9,448.4,450.6,446.2,False,1,437.303998,56.264209
4,2016-04-23 01:00:00+00:00,448.4,448.4,448.4,448.4,450.6,446.2,False,1,437.430908,56.264209
...,...,...,...,...,...,...,...,...,...,...,...
5796,2016-12-30 19:00:00+00:00,960.0,963.7,960.0,963.1,967.9,958.3,False,1,917.506133,55.356060
5797,2016-12-30 20:00:00+00:00,963.1,964.6,961.1,962.8,967.6,958.0,False,-1,917.956818,54.996099
5798,2016-12-30 21:00:00+00:00,962.8,964.3,954.0,955.5,960.2,950.8,False,-1,918.330382,46.989034
5799,2016-12-30 22:00:00+00:00,955.5,965.4,954.6,963.9,968.7,959.1,True,1,918.783811,55.091404


In [117]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE               



In [118]:
df_slim['signal'] = df_slim.apply(apply_signal, axis=1)

In [119]:
df_slim['signal'].value_counts()

 0    5443
-1     194
 1     164
Name: signal, dtype: int64

In [120]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5


def apply_take_profit(row):
    if row.signal != NONE:
        return (row.mid_c -row.mid_o) * PROFIT_FACTOR + row.mid_c
    else:
        return 0.0


def apply_stop_loss(row):
    if row.signal != NONE:
        return row.mid_o
    else:
        return 0.0

In [121]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [122]:
df_slim[df_slim.signal == SELL].head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,signal,TP,SL
132,2016-04-28 09:00:00+00:00,444.7,444.7,442.7,442.7,444.9,440.5,True,-1,452.564685,32.579723,-1,439.70,444.7
137,2016-04-28 14:00:00+00:00,447.0,447.0,445.7,445.8,448.0,443.6,True,-1,452.224528,42.185333,-1,444.00,447.0
229,2016-05-02 10:00:00+00:00,446.7,446.7,445.1,445.1,447.3,442.9,True,-1,451.941983,24.068101,-1,442.70,446.7
233,2016-05-02 14:00:00+00:00,446.1,446.1,441.5,442.3,444.5,440.1,True,-1,451.645024,24.793700,-1,436.60,446.1
237,2016-05-02 18:00:00+00:00,444.2,444.9,443.5,443.5,445.7,441.3,True,-1,451.337413,31.930613,-1,442.45,444.2


In [123]:
df_plot = df_slim.copy()

cp = CandlePlotting(df_plot, candele=True)

trades = cp.df_plot[df_plot.signal != NONE]



In [124]:
markers = ['mid_c', 'TP', 'SL']
marker_color = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
            x=trades.sTime,
            y=trades[markers[i]], 
            mode='markers', 
            marker=dict(color=marker_color[i], size = 12)
            ))

cp.show_plot(line_traces=['EMA_200'], sec_traces=['RSI_14'])

In [125]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.signal
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)  

In [126]:
open_trades = []

closed_trades = []


for index, row in trades.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
        
    open_trades = [x for x in open_trades if x.running == True]


    if row.signal != NONE:
        open_trades.append(Trade(row))
        



In [127]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [128]:
df_results.result.sum()

73.0

In [129]:
df_results.sort_values(by="start_index", inplace=True)


In [130]:
df_m5 = pd.read_pickle("../data/BTC_USD_M5.pkl")


In [131]:
df_m5.shape

(96040, 14)

In [132]:
df_m5.time.max()

Timestamp('2016-12-30 23:55:00+0000', tz='tzutc()')

In [133]:
df_raw.time.max()

Timestamp('2016-12-30 23:00:00+0000', tz='tzutc()')

In [134]:
from dateutil import parser


In [135]:
time_min = parser.parse("2016-12-15T10:00:00Z")
time_max = parser.parse("2016-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time >= time_min) & (df_m5.time <= time_max)]
df_raw_s = df_raw[(df_raw.time >= time_min) & (df_raw.time <= time_max)]

In [136]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
91619,2016-12-15 10:00:00+00:00,123,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91620,2016-12-15 10:05:00+00:00,120,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91621,2016-12-15 10:10:00+00:00,104,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91622,2016-12-15 10:15:00+00:00,95,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91623,2016-12-15 10:20:00+00:00,149,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91624,2016-12-15 10:25:00+00:00,148,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91625,2016-12-15 10:30:00+00:00,82,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91626,2016-12-15 10:35:00+00:00,103,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91627,2016-12-15 10:40:00+00:00,64,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9
91628,2016-12-15 10:45:00+00:00,202,776.1,776.1,776.1,776.1,772.3,772.3,772.3,772.3,779.9,779.9,779.9,779.9


In [137]:
df_raw_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
7724,2016-12-15 10:00:00+00:00,1717,776.1,776.7,776.1,776.7,772.3,772.9,772.3,772.9,779.9,780.5,779.9,780.5
7725,2016-12-15 11:00:00+00:00,1770,776.7,776.7,776.7,776.7,772.9,772.9,772.9,772.9,780.5,780.5,780.5,780.5


In [138]:
df_m5_slim = df_m5[['time', 'mid_h', 'mid_l']].copy()

In [139]:
df_m5_slim.tail()

,time,mid_h,mid_l
96035,2016-12-30 23:35:00+00:00,964.4,964.1
96036,2016-12-30 23:40:00+00:00,965.4,963.4
96037,2016-12-30 23:45:00+00:00,966.4,962.6
96038,2016-12-30 23:50:00+00:00,966.4,963.6
96039,2016-12-30 23:55:00+00:00,966.0,964.1


In [142]:
df_signal = df_slim[df_slim.signal != NONE].copy() 

In [145]:
df_signal['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signal.time]

In [146]:
df_signal['start_index_h1'] = df_signal.index

In [147]:
df_signal.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,ENGULFING,direction,EMA_200,RSI_14,signal,TP,SL,m5_start,start_index_h1
24,2016-04-23 21:00:00+00:00,448.2,449.9,448.2,449.9,452.1,447.7,True,1,439.837849,56.104105,1,452.45,448.2,2016-04-23 22:00:00+00:00,24
55,2016-04-25 04:00:00+00:00,459.2,459.6,459.2,459.6,461.8,457.4,True,1,444.524221,59.428622,1,460.20,459.2,2016-04-25 05:00:00+00:00,55
67,2016-04-25 16:00:00+00:00,459.2,461.7,459.0,461.7,464.0,459.4,True,1,446.283653,60.054151,1,465.45,459.2,2016-04-25 17:00:00+00:00,67
71,2016-04-25 20:00:00+00:00,462.4,466.1,462.4,466.1,468.4,463.8,True,1,447.015599,66.697305,1,471.65,462.4,2016-04-25 21:00:00+00:00,71
80,2016-04-26 05:00:00+00:00,464.2,465.5,464.2,465.5,467.8,463.2,True,1,448.667027,60.051253,1,467.45,464.2,2016-04-26 06:00:00+00:00,80


In [150]:
df_signal.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c', 'bid_c',
       'ENGULFING', 'direction', 'EMA_200', 'RSI_14', 'signal', 'TP', 'SL',
       'm5_start', 'start_index_h1'],
      dtype='object')

In [152]:
df_signal.drop(['time', 'mid_o', 'mid_h', 'mid_l','ask_c', 'bid_c','ENGULFING', 'EMA_200', 'RSI_14'], axis=1, inplace=True)

In [153]:
df_signal.head()

,mid_c,direction,signal,TP,SL,m5_start,start_index_h1
24,449.9,1,1,452.45,448.2,2016-04-23 22:00:00+00:00,24
55,459.6,1,1,460.20,459.2,2016-04-25 05:00:00+00:00,55
67,461.7,1,1,465.45,459.2,2016-04-25 17:00:00+00:00,67
71,466.1,1,1,471.65,462.4,2016-04-25 21:00:00+00:00,71
80,465.5,1,1,467.45,464.2,2016-04-26 06:00:00+00:00,80


In [158]:
df_signal.rename(columns={
        'mid_c': 'start_price',
        'm5_start': 'time',
        'signal': 'SIGNAL',
        }, inplace=True)

In [161]:
df_signal.head(2)

,start_price,direction,SIGNAL,TP,SL,time,start_index_h1
24,449.9,1,1,452.45,448.2,2016-04-23 22:00:00+00:00,24
55,459.6,1,1,460.20,459.2,2016-04-25 05:00:00+00:00,55


In [163]:
df_m5_slim.head(2)

,time,mid_h,mid_l
0,2016-01-07 00:00:00+00:00,430.6,430.2
1,2016-01-07 00:05:00+00:00,430.8,430.4


In [165]:
merged = pd.merge(left=df_m5_slim, right=df_signal, how='left', on='time')

In [170]:

merged[merged.SIGNAL.isna() == False].head()
merged.fillna(0, inplace=True)

In [173]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)


In [174]:
merged.head()

,time,mid_h,mid_l,start_price,direction,SIGNAL,TP,SL,start_index_h1
0,2016-01-07 00:00:00+00:00,430.6,430.2,0.0,0.0,0,0.0,0.0,0
1,2016-01-07 00:05:00+00:00,430.8,430.4,0.0,0.0,0,0.0,0.0,0
2,2016-01-07 00:10:00+00:00,431.6,430.4,0.0,0.0,0,0.0,0.0,0
3,2016-01-07 00:15:00+00:00,430.6,430.4,0.0,0.0,0,0.0,0.0,0
4,2016-01-07 00:20:00+00:00,431.1,430.5,0.0,0.0,0,0.0,0.0,0
